In [1]:
import numpy as np
import os
import cv2
import keras
import sklearn
import pandas
from time import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import to_categorical
from keras.models import load_model
from keras.layers import *
from keras import layers
from keras import Model
from keras import optimizers
from keras import regularizers
from keras.callbacks import TensorBoard
from keras import optimizers
import matplotlib.pyplot as plt
from keras.applications import *
from sklearn.metrics import classification_report
import time

In [2]:
feature = np.load("resnet_feature.npy")
print(feature.shape)

(30787, 7, 7, 2048)


In [3]:
violent_features = feature[0:13410]
non_violent_features = feature[-13410:]
print("Violent features: ", violent_features.shape)
print("Non Violent features: ", non_violent_features.shape)

Violent features:  (13410, 7, 7, 2048)
Non Violent features:  (13410, 7, 7, 2048)


In [32]:
Test_array = []

Test_array.append(np.array(violent_features[0:30]))
Test_array.append(np.array(violent_features[30:60]))
Test_array.append(np.array(violent_features[60:90]))
test_array = np.asarray(Test_array)
print(test_array.shape)

(3, 30, 7, 7, 2048)


In [4]:
violent_vid = []
non_violent_vid = []

i = 0
while i < len(violent_features):
    violent_vid.append(np.array(violent_features[i:i+30]))
    i = i+30

i = 0
while i < len(non_violent_features):
    non_violent_vid.append(np.array(non_violent_features[i:i+30]))
    i = i+30

violent_vid = np.asarray(violent_vid)
non_violent_vid = np.asarray(non_violent_vid)

In [40]:
violent_vid.shape

(447, 30, 7, 7, 2048)

In [5]:
violent_y, non_violent_y = np.zeros(len(violent_vid)), np.ones(len(non_violent_vid))

In [6]:
#Special Case for this paper only. 
#Violent = 0
#Non violent = 1

print("Violent Video Seq: ", violent_vid.shape,"Non_violent video Seq: ", non_violent_vid.shape)
print("Violent Label: ", violent_y.shape, "Non_violent Label: ", non_violent_y.shape)

Violent Video Seq:  (447, 30, 7, 7, 2048) Non_violent video Seq:  (447, 30, 7, 7, 2048)
Violent Label:  (447,) Non_violent Label:  (447,)


In [7]:
X = np.vstack((violent_vid,non_violent_vid))
y = np.append(violent_y, non_violent_y)
X = np.reshape(X,(X.shape[0],X.shape[1],X.shape[2]*X.shape[3]*X.shape[4]))
print(X.shape)
print(y.shape)

(894, 30, 100352)
(894,)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=0.2)
print(X_train.shape,X_test.shape,y_train.shape, y_test.shape)

(715, 30, 100352) (179, 30, 100352) (715,) (179,)


In [10]:
model = Sequential()
model.add(tf.compat.v1.keras.layers.CuDNNLSTM(50, input_shape=(X.shape[1],X.shape[2]), return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm (CuDNNLSTM)       (None, 50)                20080800  
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 20,080,851
Trainable params: 20,080,851
Non-trainable params: 0
_________________________________________________________________


In [11]:
optimizer = optimizers.Adam(lr=0.001,decay=0.004)
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])


In [12]:
start = time.time()
model.fit(X_train,y_train, epochs=20, verbose=1, validation_data=(X_test,y_test), batch_size=16)
end = time.time()

Epoch 1/20
45/45 [==============================] - 13s 137ms/step - loss: 3.2927 - accuracy: 0.5795 - val_loss: 1.5082 - val_accuracy: 0.5922
Epoch 2/20
45/45 [==============================] - 6s 125ms/step - loss: 1.2597 - accuracy: 0.7263 - val_loss: 1.0025 - val_accuracy: 0.8827
Epoch 3/20
45/45 [==============================] - 6s 124ms/step - loss: 0.9304 - accuracy: 0.8852 - val_loss: 0.8563 - val_accuracy: 0.8883
Epoch 4/20
45/45 [==============================] - 6s 126ms/step - loss: 0.7983 - accuracy: 0.9161 - val_loss: 0.7433 - val_accuracy: 0.9050
Epoch 5/20
45/45 [==============================] - 6s 125ms/step - loss: 0.7139 - accuracy: 0.9461 - val_loss: 0.6489 - val_accuracy: 0.9497
Epoch 6/20
45/45 [==============================] - 6s 126ms/step - loss: 0.6596 - accuracy: 0.9413 - val_loss: 0.6536 - val_accuracy: 0.9553
Epoch 7/20
45/45 [==============================] - 6s 126ms/step - loss: 0.6438 - accuracy: 0.9560 - val_loss: 0.5622 - val_accuracy: 0.9553
Epoch

In [14]:
model.save('resnet_LSMT_model.h5', overwrite= True, include_optimizer=True)
model.save_weights("resnet_LSTM_weights.h5", overwrite=True)